In [2]:
import fireducks.pandas as pd
import os

# Path to the extracted chunk files (from your Kaggle dataset structure)
extracted_chunks_path = "/kaggle/input/processed-chunks-1"  # Adjust if the path differs

# Combine all chunk files
all_chunks = []
for file_name in sorted(os.listdir(extracted_chunks_path)):  # Ensure files are combined in order
    if file_name.startswith("processed_chunk_") and file_name.endswith(".csv"):
        file_path = os.path.join(extracted_chunks_path, file_name)
        print(f"Loading {file_name}...")
        chunk = pd.read_csv(file_path)
        all_chunks.append(chunk)

# Concatenate all chunks into a single DataFrame
combined_data = pd.concat(all_chunks, ignore_index=True)

# Display combined data info
print("Combined data shape:", combined_data.shape)


Loading processed_chunk_0_50000.csv...
Loading processed_chunk_1000000_1050000.csv...
Loading processed_chunk_100000_150000.csv...
Loading processed_chunk_1050000_1100000.csv...
Loading processed_chunk_1100000_1150000.csv...
Loading processed_chunk_1150000_1200000.csv...
Loading processed_chunk_1200000_1250000.csv...
Loading processed_chunk_1250000_1300000.csv...
Loading processed_chunk_1300000_1350000.csv...
Loading processed_chunk_1350000_1400000.csv...
Loading processed_chunk_1400000_1450000.csv...
Loading processed_chunk_1450000_1500000.csv...
Loading processed_chunk_1500000_1550000.csv...
Loading processed_chunk_150000_200000.csv...
Loading processed_chunk_1550000_1600000.csv...
Loading processed_chunk_1600000_1650000.csv...
Loading processed_chunk_1650000_1700000.csv...
Loading processed_chunk_1700000_1750000.csv...
Loading processed_chunk_1750000_1800000.csv...
Loading processed_chunk_1800000_1850000.csv...
Loading processed_chunk_1850000_1900000.csv...
Loading processed_chunk_1

In [3]:
# Save the combined dataset as a CSV for future use
combined_data_path = "/kaggle/working/combined_processed_data.csv"
combined_data.to_csv(combined_data_path, index=False)
print(f"Combined data saved at: {combined_data_path}")
# Load the saved combined dataset
combined_data = pd.read_csv("/kaggle/working/combined_processed_data.csv")

# Check the dataset structure
print(combined_data.info())
print(combined_data.head())
# Check label distribution
print(combined_data['label'].value_counts())

Combined data saved at: /kaggle/working/combined_processed_data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600000 entries, 0 to 3599999
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   review          object
 1   label           int64 
 2   cleaned_review  object
dtypes: int64(1), object(2)
memory usage: 82.4+ MB
None
                                              review  label  \
0  Stuning even for the non-gamer: This sound tra...      2   
1  The best soundtrack ever to anything.: I'm rea...      2   
2  Amazing!: This soundtrack is my favorite music...      2   
3  Excellent Soundtrack: I truly like this soundt...      2   
4  Remember, Pull Your Jaw Off The Floor After He...      2   

                                      cleaned_review  
0  stun non gamer sound track beautiful paint sen...  
1  good soundtrack read lot review say good game ...  
2  amazing soundtrack favorite music time hand in...  
3  excellent soundtrack t

In [4]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X = combined_data['cleaned_review']  # Features (cleaned reviews)
y = combined_data['label']           # Labels (1 for neutral, 2 for positive)

# Perform train-test split (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

Training samples: 2880000
Testing samples: 720000


# TF-IDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import Parallel, delayed
import pandas as pd

# Remove missing values
X_train = X_train.fillna("")
X_test = X_test.fillna("")

# Preprocess data using parallel processing
def preprocess_text(text):
    return text.lower()  # Add more preprocessing if needed

print("Starting parallel preprocessing...")
X_train_processed = Parallel(n_jobs=-1)(delayed(preprocess_text)(text) for text in X_train)
X_test_processed = Parallel(n_jobs=-1)(delayed(preprocess_text)(text) for text in X_test)
print("Preprocessing complete!")

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=50000,  # Limit vocab size
    stop_words='english',  # Remove common words
    ngram_range=(1,2),  # Use unigrams and bigrams
    sublinear_tf=True,  # Smooth scaling for better accuracy
    max_df=0.95,  # Ignore very frequent words
    min_df=5  # Ignore rare words appearing in <5 docs
)

# Fit and transform TF-IDF on training data, transform on test data
print("Starting TF-IDF transformation...")
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_processed)
X_test_tfidf = tfidf_vectorizer.transform(X_test_processed)
print("TF-IDF transformation complete!")

# Check shape
print(f"Train TF-IDF shape: {X_train_tfidf.shape}")
print(f"Test TF-IDF shape: {X_test_tfidf.shape}")

Starting parallel preprocessing...
Preprocessing complete!
Starting TF-IDF transformation...
TF-IDF transformation complete!
Train TF-IDF shape: (2880000, 50000)
Test TF-IDF shape: (720000, 50000)


# All in one

In [6]:
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import KFold
from scipy.stats import ttest_rel
import psutil
import multiprocessing

In [ ]:

# --- Six-Model Evaluation ---
# Function to check memory usage
def check_memory():
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

# Ensure labels are numeric and 0-based
# Assuming y_train, y_test are defined (e.g., from your dataset)
y_train = np.array([int(label) for label in y_train])
y_test = np.array([int(label) for label in y_test])
y_train_adjusted = y_train - 1 if np.min(y_train) > 0 else y_train
y_test_adjusted = y_test - 1 if np.min(y_test) > 0 else y_test

# Define models
N_CORES = multiprocessing.cpu_count() - 1
models = {
    'SVM': LinearSVC(C=1.0, max_iter=5000, dual=False, random_state=42),
    'Logistic Regression': LogisticRegression(C=1.0, max_iter=5000, n_jobs=N_CORES, random_state=42),
    'Gaussian NB': GaussianNB(),
    'MLP Classifier': MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, learning_rate_init=0.001, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=200, max_depth=5, learning_rate=0.1, n_jobs=N_CORES, use_label_encoder=False, eval_metric='logloss', tree_method='hist', random_state=42),
    'LightGBM': lgb.LGBMClassifier(n_estimators=200, max_depth=5, n_jobs=N_CORES, random_state=42, verbose=-1)
}

# Step 1: Train models and find top 2 by accuracy
accuracies = {}
print("Training and evaluating models on test set...")
for name, model in models.items():
    subset_size = min(50000, X_train_tfidf.shape[0])
    # Convert to dense for Gaussian NB only
    if name == 'Gaussian NB':
        model.fit(X_train_tfidf[:subset_size].toarray(), y_train_adjusted[:subset_size])
        y_pred = model.predict(X_test_tfidf.toarray())
    else:
        model.fit(X_train_tfidf[:subset_size], y_train_adjusted[:subset_size])
        y_pred = model.predict(X_test_tfidf)
    if np.min(y_train) > 0:
        y_pred = y_pred + 1
    acc = accuracy_score(y_test, y_pred)
    accuracies[name] = acc
    print(f"{name} Accuracy: {acc:.4f}")
check_memory()

# Identify top 2 models
top_models = sorted(accuracies.items(), key=lambda x: x[1], reverse=True)[:2]
print("\nTop 2 models by accuracy:")
for name, acc in top_models:
    print(f"{name}: {acc:.4f}")

# Step 2: 10-Fold Cross-Validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
metrics = ['accuracy', 'f1', 'roc_auc']
results = {name: {metric: [] for metric in metrics} for name in models.keys()}

print("\nPerforming 10-fold cross-validation...")
for name, model in models.items():
    print(f"Cross-validating {name}...")
    for train_idx, val_idx in kf.split(X_train_tfidf):
        X_train_fold, X_val_fold = X_train_tfidf[train_idx], X_train_tfidf[val_idx]
        y_train_fold, y_val_fold = y_train_adjusted[train_idx], y_train_adjusted[val_idx]
        # Convert to dense for Gaussian NB only
        if name == 'Gaussian NB':
            model.fit(X_train_fold[:subset_size].toarray(), y_train_fold[:subset_size])
            y_pred_fold = model.predict(X_val_fold.toarray())
            # ROC-AUC not computed for Gaussian NB due to sparse-to-dense conversion
            y_prob_fold = model.predict_proba(X_val_fold.toarray())[:, 1] if hasattr(model, "predict_proba") else None
        else:
            model.fit(X_train_fold[:subset_size], y_train_fold[:subset_size])
            y_pred_fold = model.predict(X_val_fold)
            y_prob_fold = model.predict_proba(X_val_fold)[:, 1] if hasattr(model, "predict_proba") else None
        
        if np.min(y_train) > 0:
            y_pred_fold = y_pred_fold + 1
            y_val_fold = y_val_fold + 1
        
        results[name]['accuracy'].append(accuracy_score(y_val_fold, y_pred_fold))
        results[name]['f1'].append(f1_score(y_val_fold, y_pred_fold, average='weighted'))
        if y_prob_fold is not None:
            results[name]['roc_auc'].append(roc_auc_score(y_val_fold, y_prob_fold))
        else:
            results[name]['roc_auc'].append(np.nan)

# Format cross-validation results
print("\n### 1. Perform K-Fold Cross-Validation and Report Detailed Performance Metrics")
print("Table: Performance of Models with TF-IDF Features (10-Fold Cross-Validation)")
print("| Model              | Accuracy (Mean ± SD) | F1-Score (Mean ± SD) | AUC-ROC (Mean ± SD) |")
print("|--------------------|---------------------|---------------------|--------------------|")
for name in models.keys():
    acc_mean, acc_std = np.mean(results[name]['accuracy']), np.std(results[name]['accuracy'])
    f1_mean, f1_std = np.mean(results[name]['f1']), np.std(results[name]['f1'])
    auc_mean, auc_std = np.mean(results[name]['roc_auc']), np.std(results[name]['roc_auc'])
    print(f"| {name:<18} | {acc_mean:.2f} ± {acc_std:.2f}         | {f1_mean:.2f} ± {f1_std:.2f}         | {auc_mean:.2f} ± {auc_std:.2f}        |")

# Step 3: Statistical Significance Tests
top_model_1_name, top_model_2_name = top_models[0][0], top_models[1][0]
top_model_1_scores = results[top_model_1_name]['accuracy']
top_model_2_scores = results[top_model_2_name]['accuracy']

print("\n### 2. Conduct Statistical Significance Tests and Present p-Values")
print(f"Table: Statistical Significance of {top_model_1_name} and {top_model_2_name} vs. Other Models (TF-IDF)")
print("| Comparison                  | p-value (Accuracy) | Significant? |")
print("|-----------------------------|-------------------|--------------|")
for name in models.keys():
    if name != top_model_1_name:
        other_scores = results[name]['accuracy']
        t_stat, p_value = ttest_rel(top_model_1_scores, other_scores)
        significant = "Yes" if p_value < 0.05 else "No"
        print(f"| {top_model_1_name} vs. {name:<15} | {p_value:.3f}             | {significant:<12} |")
for name in models.keys():
    if name != top_model_2_name and name != top_model_1_name:
        other_scores = results[name]['accuracy']
        t_stat, p_value = ttest_rel(top_model_2_scores, other_scores)
        significant = "Yes" if p_value < 0.05 else "No"
        print(f"| {top_model_2_name} vs. {name:<15} | {p_value:.3f}             | {significant:<12} |")
check_memory()

Training and evaluating models on test set...
SVM Accuracy: 0.8683
Logistic Regression Accuracy: 0.8735


# LightGBM

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
import psutil
import numpy as np

# Function to check memory usage
def check_memory():
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

# Ensure labels are 0-based
y_train_adjusted = np.array(y_train) - 1
y_test_adjusted = np.array(y_test) - 1

# Define LightGBM hyperparameters with reduced iterations and depth
param_grid = {
    'learning_rate': [0.01, 0.05],
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'num_leaves': [15, 31]
}

# Stratified K-Fold Cross-Validation (using 3 folds for efficiency)
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Iterate over hyperparameters
for lr in param_grid['learning_rate']:
    for iterations in param_grid['n_estimators']:
        for depth in param_grid['max_depth']:
            for num_leaves in param_grid['num_leaves']:
                print(f"Training LightGBM with learning_rate={lr}, iterations={iterations}, depth={depth}, num_leaves={num_leaves}...")
                
                # Initialize the LightGBM model
                lgbm_classifier = lgb.LGBMClassifier(
                    learning_rate=lr,
                    n_estimators=iterations,
                    max_depth=depth,
                    num_leaves=num_leaves,
                    verbose=-1,
                    n_jobs=-1  # Utilize all CPU cores
                )
                
                # Train on a subset for efficiency
                subset_size = min(10000, X_train_tfidf.shape[0])
                lgbm_classifier.fit(X_train_tfidf[:subset_size], y_train_adjusted[:subset_size])
                
                # Make predictions
                y_pred_lgbm = lgbm_classifier.predict(X_test_tfidf) + 1  # Adjust back to original label scale
                
                # Performance evaluation
                print(f"LightGBM Classification Report (learning_rate={lr}, iterations={iterations}, depth={depth}, num_leaves={num_leaves}):")
                print(classification_report(y_test, y_pred_lgbm))
                print(f"LightGBM Accuracy: {accuracy_score(y_test, y_pred_lgbm):.4f}\n")
                
                # Check memory usage
                check_memory()

# CatBoost

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
import psutil

# Function to check memory usage
def check_memory():
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

# Ensure labels are 0-based
y_train_adjusted = np.array(y_train) - 1
y_test_adjusted = np.array(y_test) - 1

# Define hyperparameter grid for CatBoost
param_grid = {
    'learning_rate': [0.01, 0.05],
    'iterations': [50, 100],
    'depth': [3, 5],
    'l2_leaf_reg': [1, 3],
    'border_count': [32, 50]
}

# Stratified K-Fold Cross-Validation (using 3 folds for efficiency)
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Iterate over hyperparameters
for lr in param_grid['learning_rate']:
    for iterations in param_grid['iterations']:
        for depth in param_grid['depth']:
            for l2_leaf_reg in param_grid['l2_leaf_reg']:
                for border_count in param_grid['border_count']:
                    print(f"Training CatBoost with learning_rate={lr}, iterations={iterations}, depth={depth}, l2_leaf_reg={l2_leaf_reg}, border_count={border_count}...")
                    
                    # Initialize the CatBoost model
                    catboost_classifier = CatBoostClassifier(
                        learning_rate=lr,
                        iterations=iterations,
                        depth=depth,
                        l2_leaf_reg=l2_leaf_reg,
                        border_count=border_count,
                        verbose=0,
                        random_state=42
                    )
                    
                    # Train on a subset for efficiency
                    subset_size = min(10000, X_train_tfidf.shape[0])
                    catboost_classifier.fit(X_train_tfidf[:subset_size], y_train_adjusted[:subset_size])
                    
                    # Make predictions
                    y_pred_catboost = catboost_classifier.predict(X_test_tfidf) + 1  # Adjust back to original label scale
                    
                    # Performance evaluation
                    print(f"CatBoost Classification Report (learning_rate={lr}, iterations={iterations}, depth={depth}, l2_leaf_reg={l2_leaf_reg}, border_count={border_count}):")
                    print(classification_report(y_test, y_pred_catboost))
                    print(f"CatBoost Accuracy: {accuracy_score(y_test, y_pred_catboost):.4f}\n")
                    
                    # Check memory usage
                    check_memory()

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
import psutil
import numpy as np

# Function to check memory usage
def check_memory():
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

# Ensure labels are 0-based
y_train_adjusted = np.array(y_train) - 1
y_test_adjusted = np.array(y_test) - 1

# Define hyperparameter grid for RandomForest
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5]
}

# Stratified K-Fold Cross-Validation (using 3 folds for efficiency)
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Iterate over hyperparameters
for n_estimators in param_grid['n_estimators']:
    for depth in param_grid['max_depth']:
        for min_samples_split in param_grid['min_samples_split']:
            print(f"Training Random Forest with n_estimators={n_estimators}, depth={depth}, min_samples_split={min_samples_split}...")
            
            # Initialize the RandomForest model
            rf_classifier = RandomForestClassifier(
                n_estimators=n_estimators,
                max_depth=depth,
                min_samples_split=min_samples_split,
                n_jobs=-1,  # Utilize all CPU cores
                random_state=42
            )
            
            # Train on a subset for efficiency
            subset_size = min(10000, X_train_tfidf.shape[0])
            rf_classifier.fit(X_train_tfidf[:subset_size], y_train_adjusted[:subset_size])
            
            # Make predictions
            y_pred_rf = rf_classifier.predict(X_test_tfidf) + 1  # Adjust back to original label scale
            
            # Performance evaluation
            print(f"Random Forest Classification Report (n_estimators={n_estimators}, depth={depth}, min_samples_split={min_samples_split}):")
            print(classification_report(y_test, y_pred_rf))
            print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}\n")
            
            # Check memory usage
            check_memory()

# AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
import psutil
import numpy as np

# Function to check memory usage
def check_memory():
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

# Ensure labels are 0-based
y_train_adjusted = np.array(y_train) - 1
y_test_adjusted = np.array(y_test) - 1

# Define hyperparameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1]
}

# Stratified K-Fold Cross-Validation (using 3 folds for efficiency)
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Iterate over hyperparameters
for n_estimators in param_grid['n_estimators']:
    for learning_rate in param_grid['learning_rate']:
        print(f"Training AdaBoost with n_estimators={n_estimators}, learning_rate={learning_rate}...")
        
        # Initialize the AdaBoost model
        adaboost_classifier = AdaBoostClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            random_state=42
        )
        
        # Train on a subset for efficiency
        subset_size = min(10000, X_train_tfidf.shape[0])
        adaboost_classifier.fit(X_train_tfidf[:subset_size], y_train_adjusted[:subset_size])
        
        # Make predictions
        y_pred_adaboost = adaboost_classifier.predict(X_test_tfidf) + 1  # Adjust back to original label scale
        
        # Performance evaluation
        print(f"AdaBoost Classification Report (n_estimators={n_estimators}, learning_rate={learning_rate}):")
        print(classification_report(y_test, y_pred_adaboost))
        print(f"AdaBoost Accuracy: {accuracy_score(y_test, y_pred_adaboost):.4f}\n")
        
        # Check memory usage
        check_memory()


# Naive Bayes and SVM

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import psutil

# Function to check memory usage
def check_memory():
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

# Ensure labels are 0-based
y_train_adjusted = np.array(y_train) - 1
y_test_adjusted = np.array(y_test) - 1

# Naive Bayes hyperparameter tuning
alpha_values = [0.1, 0.5, 1.0, 2.0]
for alpha in alpha_values:
    print(f"Training Naive Bayes with alpha={alpha}...")
    nb_classifier = MultinomialNB(alpha=alpha)
    nb_classifier.fit(X_train_tfidf, y_train_adjusted)
    
    # Predict
    y_pred_nb = nb_classifier.predict(X_test_tfidf) + 1  # Adjust back to original labels
    
    # Performance evaluation
    print(f"Naive Bayes Classification Report (alpha={alpha}):")
    print(classification_report(y_test, y_pred_nb))
    print(f"Naive Bayes Accuracy: {accuracy_score(y_test, y_pred_nb):.4f}\n")
    check_memory()

# SVM hyperparameter tuning
c_values = [0.1, 0.5, 1.0, 2.0]
for c_value in c_values:
    print(f"Training SVM with C={c_value}...")
    svm_classifier = LinearSVC(C=c_value, max_iter=5000, random_state=42)
    svm_classifier.fit(X_train_tfidf, y_train_adjusted)
    
    # Predict
    y_pred_svm = svm_classifier.predict(X_test_tfidf) + 1  # Adjust back to original labels
    
    # Performance evaluation
    print(f"SVM Classification Report (C={c_value}):")
    print(classification_report(y_test, y_pred_svm))
    print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}\n")
    check_memory()

# MLP Classifier

In [ ]:
# from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import classification_report, accuracy_score
# import numpy as np
# import psutil

# # Function to check memory usage
# def check_memory():
#     print(f"Memory usage: {psutil.virtual_memory().percent}%")

# # Ensure labels are 0-based
# y_train_adjusted = np.array(y_train) - 1
# y_test_adjusted = np.array(y_test) - 1

# # Define hyperparameter grid for MLP
# hidden_layer_sizes_values = [(100,), (128, 64)]
# activation_values = ['relu', 'tanh']
# solver_values = ['adam', 'sgd']

# # Iterate over hyperparameters
# for hidden_layer_sizes in hidden_layer_sizes_values:
#     for activation in activation_values:
#         for solver in solver_values:
#             print(f"Training MLPClassifier with hidden_layer_sizes={hidden_layer_sizes}, activation={activation}, solver={solver}...")

#             # Set up the MLPClassifier model with the specified hyperparameters
#             mlp_classifier = MLPClassifier(
#                 hidden_layer_sizes=hidden_layer_sizes,
#                 activation=activation,
#                 solver=solver,
#                 max_iter=300,
#                 random_state=42
#             )

#             # Train the model
#             mlp_classifier.fit(X_train_tfidf[:50000], y_train_adjusted[:50000])  # Use a larger subset
#             y_pred_mlp = mlp_classifier.predict(X_test_tfidf) + 1  # Adjust back to original labels

#             # Performance reporting
#             print(f"MLPClassifier Classification Report for hidden_layer_sizes={hidden_layer_sizes}, activation={activation}, solver={solver}:")
#             print(classification_report(y_test, y_pred_mlp))
#             print(f"MLPClassifier Accuracy: {accuracy_score(y_test, y_pred_mlp):.4f}\n")

#             # Check memory usage after each iteration
#             check_memory()

# XG Boost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
import psutil

# Function to check memory usage
def check_memory():
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

# Ensure labels are 0-based
y_train_adjusted = np.array(y_train) - 1
y_test_adjusted = np.array(y_test) - 1

# Define hyperparameter grid for XGBoost
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'min_child_weight': [1, 5],
    'gamma': [0, 0.1]
}

# Stratified K-Fold Cross-Validation (using 3 folds for efficiency)
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Iterate over hyperparameters
for lr in param_grid['learning_rate']:
    for n_est in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for min_child_weight in param_grid['min_child_weight']:
                for gamma in param_grid['gamma']:
                    print(f"Training XGBoost with learning_rate={lr}, n_estimators={n_est}, max_depth={max_depth}, min_child_weight={min_child_weight}, gamma={gamma}...")
                    
                    # Initialize the XGBoost model
                    xgb_classifier = XGBClassifier(
                        learning_rate=lr,
                        n_estimators=n_est,
                        max_depth=max_depth,
                        min_child_weight=min_child_weight,
                        gamma=gamma,
                        use_label_encoder=False,
                        eval_metric='logloss',
                        tree_method='hist',
                        random_state=42
                    )
                    
                    # Train on a subset for efficiency
                    subset_size = min(10000, X_train_tfidf.shape[0])
                    xgb_classifier.fit(X_train_tfidf[:subset_size], y_train_adjusted[:subset_size])
                    
                    # Make predictions
                    y_pred_xgb = xgb_classifier.predict(X_test_tfidf) + 1  # Adjust back to original label scale
                    
                    # Performance evaluation
                    print(f"XGBoost Classification Report (learning_rate={lr}, n_estimators={n_est}, max_depth={max_depth}, min_child_weight={min_child_weight}, gamma={gamma}):")
                    print(classification_report(y_test, y_pred_xgb))
                    print(f"XGBoost Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}\n")
                    
                    # Check memory usage
                    check_memory()